# Lecture 6 - Part II - More on Writing Julia Packages

- In the second lecture, we talked about [Pkg.jl](https://pkgdocs.julialang.org/v1/), Julia's built-in package manager.
<br>

- Today, you will see how to write your own packages, so that you can write a simple SIR package.

## Contents
1. [Pkg.jl](#pkg)
2. [PkgTemplates.jl](#pkg-templates)
3. [Modules](#modules)
4. [Registries](#registry)
5. [Tests](#tests)
6. [Documentation](#docs)

## 1. Pkg.jl <a class="anchor" id="pkg"></a>

- Hopefully, you remember the basic usage of Pkg.jl for managing enviroments and packages:

    * `activate`: activate an environment 
    * `add`, `rm`: add and remove packages from the current environment 
    * `up`: update the current environment 
    * `status`: display information about the current environment
    * `test`: test if a package works correctly on our system<br><br>

- The dependencies and the status of an environment are stored in the `Project.toml` and `Manifest.toml` files that are automatically created (and usually also edited automatically).
<br>

- We saw that we can generate a new project like this:

In [11]:
cd("/home/alistair/code/")

In [12]:
] generate MyTest

  Generating  project MyTest:
    MyTest/Project.toml
    MyTest/src/MyTest.jl


- Remember we can use `]` to execute shell commands from the Julia REPL-

In [13]:
;tree MyTest

MyTest
├── Project.toml
└── src
    └── MyTest.jl

1 directory, 2 files


In [14]:
; less MyTest/src/MyTest.jl

module MyTest

greet() = print("Hello World!")

end # module MyTest


- The Linux `less` command views the contents of a file.
<br>

- The `generate` command already gives us the basic structure of a package with a `Project.toml` for the environment and `MyTest.jl` inside the `src` folder that defines the module `MyTest`.
<br>

- An alternative and more feature-complete approach is possible with PkgTemplates.jl.

## 2. [PkgTemplates.jl](https://github.com/JuliaCI/PkgTemplates.jl) <a class="anchor" id="pkg-templates"></a>

* PkgTemplates.jl provides a more sophisticated initialization that can e.g. include GitHub, CI/CD, licences, etc. 
<br>

* It works by defining a [template](https://juliaci.github.io/PkgTemplates.jl/stable/user/#Template).
<br>

* The minimum is: 
    * The GitHub or Gitlab username `user`
    * Your name and mail `authors`
    * The directory on your local machine `dir` 
    * The minimum required Julia version `julia`

In [16]:
using PkgTemplates

template2 = Template(
    user="white-alistair",
    authors="Alistair White <alistair.white@tum.de>",
    dir=".",
    julia=v"1.9.3",
    plugins=[
        Documenter(),
    ]
)

Template:
  authors: ["Alistair White <alistair.white@tum.de>"]
  dir: "~/code"
  host: "github.com"
  julia: v"1.9.3"
  user: "white-alistair"
  plugins:
    CompatHelper:
      file: "~/.julia/packages/PkgTemplates/LISo2/templates/github/workflows/CompatHelper.yml"
      destination: "CompatHelper.yml"
      cron: "0 0 * * *"
    Documenter:
      assets: String[]
      logo: Logo(nothing, nothing)
      makedocs_kwargs: Dict{Symbol, Any}()
      canonical_url: nothing
      make_jl: "~/.julia/packages/PkgTemplates/LISo2/templates/docs/make.jl"
      index_md: "~/.julia/packages/PkgTemplates/LISo2/templates/docs/src/index.md"
      devbranch: nothing
      edit_link: :devbranch
    Git:
      ignore: String[]
      name: nothing
      email: nothing
      branch: "main"
      ssh: false
      jl: true
      manifest: false
      gpgsign: false
    GitHubActions:
      file: "~/.julia/packages/PkgTemplates/LISo2/templates/github/workflows/CI.yml"
      destination: "CI.yml"
      linu

In [17]:
template2("MyTestTwo.jl")

[ Info: Running prehooks
[ Info: Running hooks
  Activating project at `~/code/MyTestTwo`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/code/MyTestTwo/Project.toml`
  No Changes to `~/code/MyTestTwo/Manifest.toml`
Precompiling project...
  ✓ MyTestTwo
  1 dependency successfully precompiled in 0 seconds
  Activating project at `~/.julia/environments/v1.9`
  Activating new project at `~/code/MyTestTwo/docs`
   Resolving package versions...
    Updating `~/code/MyTestTwo/docs/Project.toml`
  [e30172f5] + Documenter v1.1.2
    Updating `~/code/MyTestTwo/docs/Manifest.toml`
  [a4c015fc] + ANSIColoredPrinters v0.0.1
  [1520ce14] + AbstractTrees v0.4.4
  [ffbed154] + DocStringExtensions v0.9.3
  [e30172f5] + Documenter v1.1.2
  [b5f81e59] + IOCapture v0.2.3
  [682c06a0] + JSON v0.21.4
  [0e77f7df] + LazilyInitializedFields v1.2.2
  [d0879d2d] + MarkdownAST v0.1.2
  [69de0a69] + Parsers v2.8.0
  [aea7be01] + PrecompileTools v1.2.0
  [21216c6a] + Preferences v1

- It's handy to save the `Template` call somewhere, e.g. directly in Julia's `startup.jl` script. See the [PkgTemplates.jl doc for different options](https://juliaci.github.io/PkgTemplates.jl/stable/user/#Saving-Templates-1).
<br>

- This creates the basic package structure: 

In [18]:
;tree MyTestTwo

MyTestTwo
├── docs
│   ├── make.jl
│   ├── Manifest.toml
│   ├── Project.toml
│   └── src
│       └── index.md
├── LICENSE
├── Manifest.toml
├── Project.toml
├── README.md
├── src
│   └── MyTestTwo.jl
└── test
    └── runtests.jl

4 directories, 10 files


- This already initialised a few more files. For example, it includes a license file:

In [20]:
;less MyTestTwo/LICENSE

MIT License

Copyright (c) 2023 Alistair White <alistair.white@tum.de>

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING

- And an almost empty README file:

In [21]:
;less MyTestTwo/README.md

# MyTestTwo

[![Build Status](https://github.com/white-alistair/MyTestTwo.jl/actions/workflows/CI.yml/badge.svg?branch=main)](https://github.com/white-alistair/MyTestTwo.jl/actions/workflows/CI.yml?query=branch%3Amain)


- We also have the folder structure for tests, which we will talk about in a few minutes. 
<br>

- Additionally, PkgTemplates.jl provides additional, more advanced features via the keyword `plugins`. 
Plugins can modify some of these files and add further template files 

    * `License(name="MIT", destination="LICENSE")` for the license file 
    * `Git(....)` for the git configuration
    * Options for CI/CD (we will talk about that in the next lectures)
    * Further Options for tests
    * `Documenter()` for documentation (can also include CI presets)
    * Badges that are displayed on the GitHub page that link to the documentation, CI report, etc.<br><br>

- See the [full documentation of PackageTemplates.jl](https://invenia.github.io/PkgTemplates.jl/dev/user/) for details of all the plugins that can be added.

## 3. Modules <a class="anchor" id="modules"></a>

* We first write a function for our module: 

In [22]:
"""
    add_two(x::T) where {T<:Number} 

Adds two to the input number.
"""
function add_two(x::T) where {T<:Number}
    x + T(2) 
end

"""
    add_two(x::AbstractArray{T,N}) where {T,N} 

Adds two to an array element-wise.
"""
function add_two(x::AbstractArray{T,N}) where {T,N}
    x .+ T(2)
end

add_two

- For the documentation that we add later, we should include docstrings. There's no clear convention wheather you should write one docstring for every function of the same name or just one for all. The docstring should cover all the behaviour of all functions though!
<br>

* We've seen the [module](https://docs.julialang.org/en/v1/manual/modules/) command before; it defines a [namespace](https://en.wikipedia.org/wiki/Namespace). It is the common practice that every package defines a module in the central `src/MyTestTwo.jl` file with a name such as `MyTestTwo`. This will usually look something like this: 

In [ ]:
module MyTestTwo 
    using StatsBase # import other packages that your package needs 

    include("more_functions.jl") # include source code files where the actual functions of your project are 
    
    export add_two # export some of the functions that the users can use directly

    function __init__() # OPTIONAL: this special function is always executed when the module is loaded 
        nothing 
    end
end 

- This module definition contains a number of important keywords that you'll see and use often:
    * `include`: This simply executes the code in the given file. The order of the files that you include matters; they are loaded sequentially, one after the other. If you e.g. define abstract types, you should do this in the first file you include (or sometimes also directly in the module file).
    <br>

    * `export`: This exports the given objects directly into the namespace of the importer. That means all objects that are exported can be accessed directly after the user imports the library with `using MyTestTwo`. All other objects can be still accessed directly with e.g. `MyTestTwo.other_function`
    <br>

    * `__init__`: The init function is usually not necessary. However, it is sometimes used to determine what hardware is available and chooses the right functions based on that (e.g. whether a GPU is available or not).

In [ ]:
;tree .

- Remember that we also need to add all packages we imported to environment.

## 4. Registries <a class="anchor" id="registry"></a>

- The Julia community manages packages in [registries](https://pkgdocs.julialang.org/v1/registries/). 

In [23]:
]registry st

Registry Status 
 [23338594] General (https://github.com/JuliaRegistries/General.git)


* Normally, that is just the  [General](https://github.com/JuliaRegistries/General) registry that is itself a GitHub repository where all packages are listed. 
<br>

* This repository is not managed manually but by bots. As long as a package follows the regular Julia package structure, it is relatively easy to add it to this registry with the [Registrator](https://github.com/JuliaRegistries/Registrator.jl/#registrator) bot.

## 5. Tests <a class="anchor" id="tests"></a>

- It is good practice to write [unit tests](https://en.wikipedia.org/wiki/Unit_testing) for as much of the functionality of your package as is possible. 
<br>

- The purpose of a unit test is, as the name suggests, to verify that a given "unit" of code in your package works as intended. Hence, unit tests should ideally isolate and test a single piece of functionality.
<br>

- The basic idea is that, if all the individual units of your code work as intended, then the overall program formed by putting them together will also work (this is, of course, a simplification; see also [integration testing](https://en.wikipedia.org/wiki/Integration_testing)).
<br>

- Unit tests are particularly useful for catching unwanted errors: you might make changes to one function that is also used by other functions and affects them. A properly written unit test makes sure that all functions still work as intended. 
<br>

- Julia uses the `Test` package for testing. In the package structure, all tests are written in the `test` subfolder that was created by `PkgTemplates` for us. These test are executed every time we call `]test MyTestTwo`. 
<br>

- Usually, each important function is tested seperately.
<br>

- One should avoid large computation in tests, i.e. tests should run fairly quickly (~ minutes).
<br>

- The `runtests.jl` files is meant to load common packages and include all different tests

In [24]:
]add Test

   Resolving package versions...
    Updating `~/.julia/environments/v1.9/Project.toml`
  [8dfed614] + Test
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
Precompiling project...
  ✓ Distributions → DistributionsChainRulesCoreExt
  ✓ DistributionsAD
  ✓ DistributionsAD → DistributionsADForwardDiffExt
  ✓ Tracker
  ✓ ArrayInterface → ArrayInterfaceTrackerExt
  ✓ Tracker → TrackerPDMatsExt
  ✓ FiniteDiff
  ✓ RecursiveArrayTools
  ✓ StaticArrayInterface
  ✓ SciMLOperators
  ✓ FiniteDiff → FiniteDiffStaticArraysExt
  ✓ NLSolversBase
  ✓ DistributionsAD → DistributionsADTrackerExt
  ✓ CloseOpenIntervals
  ✓ StaticArrayInterface → StaticArrayInterfaceOffsetArraysExt
  ✓ DataInterpolations
  ✓ LayoutPointers
  ✓ StaticArrayInterface → StaticArrayInterfaceStaticArraysExt
  ✓ RecursiveArrayTools → RecursiveArrayToolsTrackerExt
  ✓ DataInterpolations → DataInterpolationsChainRulesCoreExt
  ✓ EllipsisNotation
  ✓ SparseDiffTools
  ✓ StrideArraysCore
  ✓ LineSearches
  ✓ Polyester
  ✓ 

In [25]:
using Test 

In [ ]:
@test add_two(2.) ≈ 4.
    
@test add_two([2.,4.]) ≈ [4.,6.]

In [ ]:
using Test, MyTestTwo # import packages that all tests need 

# maybe you load some data or define some constant that all tests need 

@testset "Test MyTestTwo.jl Basics" begin
    include("function_tests.jl") # include individual tests 
end 

* A `@testset` groups different tests together
<br>

* The argument to `@test` must be a boolean; if it evaluates to `true` the test passes, if it evaluates to `false` the test fails.
<br>

* Different operating systems and hardware can sometimes lead to slightly different numerical results; it is important to account for that when writing tests with some kind of relative tolerence. In general, you should never check for exact equality of floating point numbers!
<br>

* An easy way to do this is by using `≈` (`\approx` in Julia) instead of `==`

### Random Numbers and Tests
* If your functions use random numbers at some point, it is important to make the results reproducible by specifying the [seed](https://en.wikipedia.org/wiki/Random_seed) for the random number generator.

In [27]:
rand()

0.6509715267800616

In [28]:
using Random

Random.seed!(1234)

TaskLocalRNG()

In [29]:
rand()

0.32597672886359486

In [30]:
rand()

0.5490511363155669

In [31]:
Random.seed!(1234)

TaskLocalRNG()

In [32]:
rand()

0.32597672886359486

In [33]:
rand()

0.5490511363155669

- Julia's default random number generator should produce consistent results across different operating systems when the random seed is set.

## 6. Documentation <a class="anchor" id="docs"></a>

- Every package also should have documentation, especially when it is intended to be used by other people (e.g. us when we are reviewing your projects!). 
<br>

- The Julia package structure also has a dedicated place for that as well as libraries that help to build the documentation. The most common library for that is [Documenter.jl](https://documenter.juliadocs.org/stable/).
<br>

- In fact, the package we created using PkgTemplates.jl already has much of the infrastructure we need for doing this.

In [35]:
;tree MyTestTwo

MyTestTwo
├── docs
│   ├── make.jl
│   ├── Manifest.toml
│   ├── Project.toml
│   └── src
│       └── index.md
├── LICENSE
├── Manifest.toml
├── Project.toml
├── README.md
├── src
│   ├── functions.jl
│   └── MyTestTwo.jl
└── test
    ├── functions.jl
    └── runtests.jl

4 directories, 12 files


* With Documenter.jl we can write the documentation using Markdown, similar to what we are doing here in Jupyter notebooks
<br>

* Documenter.jl also scans all source files for docstrings that are written directly before the function definitons.
<br>

* The `@autodocs` macro automatically collects all docstrings of all functions in our package. 
<br>

* The preset that we generated with `PkgTemplates` will already give as a first draft of a documentation when we build it (we have to do this outside of a Jupyter notebook).
<br>

* The documentation is saved in the `docs/build` folder.
<br>

* If you have LaTeX installed, it is also possible to get a pdf using LaTeX.
<br>

* Instead of `@autodocs` we can also directly choose ourselves which functions are listed with the `@docs` macro: 

In [ ]:
## One of our functions 

Here, we just list this one function
    
```@docs
add_two 
```

- We can also add more pages to the documentation by editing the `make.jl` file. We just have to add them to the `pages` keyword:

In [ ]:
using MyTestTwo
using Documenter

DocMeta.setdocmeta!(MyTestTwo, :DocTestSetup, :(using MyTestTwo); recursive=true)

makedocs(;
    modules=[MyTestTwo],
    authors="My Name <mail@adress.com>",
    repo="https://github.com/my-github-user-name/MyTestTwo.jl/blob/{commit}{path}#{line}",
    sitename="MyTestTwo.jl",
    format=Documenter.HTML(;
        prettyurls=get(ENV, "CI", "false") == "true",
        assets=String[],
    ),
    pages=[
        "Home" => "index.md",
        "Functions" => "functions.md"
    ],
)

- To make the documention: 
    * Open Julia and change to the `docs` folder
    * Activate the `docs` environment
    * Execute `include(make.jl)`
    
* The documentation will not fully work if you just open it in your browser unfortunately (in this case it does though). You will either need to run a local webserver (e.g. with `python3 -m http.server` and then opening `localhost:8000` in your browser) or upload it somewhere.
<br>

* In the next lectures we will look at how to automate much of what we've seen in this lecture, including uploading and hosting documentation as well as running tests with CI/CD.